In [ ]:
import os
import random
from datetime import datetime

path = "./Dataset/data"
labelPath = "Dataset/labels"

data = os.listdir(path)
datasetLen = len(data)

# random shuffle data
# random.shuffle(data)

In [ ]:
import SimpleITK as sitk
import numpy as np
from utils import *
import os

timeStep = 0.04
numberOfIterations = 10

# Set the number of threads
# sitk.ImageRegistrationMethod().SetGlobalDefaultNumberOfThreads(1)

pathName = os.path.join(path, data[0])
originalImage = sitk.ReadImage(pathName)
originalLabel = sitk.ReadImage(os.path.join(labelPath, data[0]))

curvatureImage = sitk.CurvatureFlow(originalImage, 
                                timeStep = timeStep, 
                                numberOfIterations = numberOfIterations)

pathName = os.path.join(path, data[2])
'''
originalMovingImage = sitk.ReadImage(pathName)
MovingImage = sitk.HistogramMatching(sitk.CurvatureFlow(originalMovingImage,
                                                  timeStep = timeStep, 
                                                  numberOfIterations = numberOfIterations),
                               curvatureImage)
'''
originalMovingImage = sitk.CurvatureFlow(sitk.ReadImage(pathName),
                                                  timeStep = timeStep, 
                                                  numberOfIterations = numberOfIterations)
MovingImage = sitk.HistogramMatching(originalMovingImage, curvatureImage)


In [ ]:
from PIL import Image
import numpy as np

def showImg_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = Image.fromarray(np.interp(a, (a.min(), a.max()), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg2_(img, z=60):
    disImg = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:]).astype('uint8'))
    disImg.show()
    
def showImg3_(img, z=60):
    a = sitk.GetArrayFromImage(img[z,:,:])
    disImg = image.fromarray(np.interp(a, (0, 5000), (0, 255)).astype('uint8'))
    disImg.show()
    
def showImg4_(img, z=60):
    img2 = Image.fromarray(sitk.GetArrayFromImage(img[z,:,:])*60, 'L')
    img2.show()
    
def showImg5_(img):
    a = sitk.GetArrayFromImage(img)
    disImg = Image.fromarray(np.interp(a, (a.min(), a.max()), (0, 255)).astype('uint8'))
    disImg.show()
    return disImg

def createOverlays(image1, image2):
    whiteImage = sitk.GetImageFromArray(255*np.ones(image1.GetSize()[::-1]))
    whiteImage.CopyInformation(image1)
    image1.CopyInformation(whiteImage)
    temp = sitk.LabelOverlay(whiteImage, image1, opacity=0.1)
    whiteImage2 = sitk.GetImageFromArray(255*np.ones(image2.GetSize()[::-1]))
    whiteImage2.CopyInformation(image2)
    temp2 = sitk.LabelOverlay(whiteImage2, image2, opacity=0.1, colormap=(0,0,0,1,0,0,0,1))

    overlay1 = Image.fromarray(sitk.GetArrayFromImage(temp).astype('uint8'), 'RGB')
    overlay2 = Image.fromarray(sitk.GetArrayFromImage(temp2).astype('uint8'), 'RGB')
    overlay11 = overlay1.convert("RGBA")
    overlay22 = overlay2.convert("RGBA")

    new_img = Image.blend(overlay11, overlay22, 0.5)
    
    return (sitk.GetImageFromArray(overlay11), sitk.GetImageFromArray(overlay22), sitk.GetImageFromArray(new_img))

In [ ]:
z = 60
showImg5_(originalImage[z,5:-5,:]).save('thesis/images/original_1.png')
showImg5_(curvatureImage[z,5:-5,:]).save('thesis/images/curvature_1.png')

z = 100
showImg5_(originalImage[z,150:300,50:200]).save('thesis/images/original_1_1.png')
showImg5_(curvatureImage[z,150:300,50:200]).save('thesis/images/curvature_1_1.png')

z = 250
showImg5_(originalImage[:,z,:]).save('thesis/images/original_2.png')
showImg5_(curvatureImage[:,z,:]).save('thesis/images/curvature_2.png')

z = 250
showImg5_(originalImage[:,:,z]).save('thesis/images/original_3.png')
showImg5_(curvatureImage[:,:,z]).save('thesis/images/curvature_3.png')

In [ ]:
import matplotlib.pyplot as plt

# plt.style.use('seaborn-deep')

histogramFig, ax1 = plt.subplots(dpi=300)

plt.hist(sitk.GetArrayFromImage(curvatureImage).reshape((-1,)), bins='auto', alpha=0.7, label='Fixed Image')
_, bins, _ = plt.hist(sitk.GetArrayFromImage(MovingImage).reshape((-1,)), bins='auto', alpha=0.7, label='Moving image after histogram matching')
# plt.hist(sitk.GetArrayFromImage(originalMovingImage).reshape((-1,)), bins=bins, alpha=0.7, label='Original Moving image')
plt.hist(sitk.GetArrayFromImage(originalMovingImage).reshape((-1,)), bins='auto', alpha=0.7, label='Original Moving image')

ax1.set_xlim([-1,200])

plt.legend(loc='best')
plt.title("Histograms")
plt.xlabel("Image intensity")
plt.ylabel("Frequency")

plt.show()

histogramFig.savefig('thesis/images/histogram_plot.png')

In [ ]:
z = 60
# showImg5_(curvatureImage[z,:,:]).save('thesis/images/curvature_1.png') # we got this
z = 50
showImg5_(originalMovingImage[z,:,:]).save('thesis/images/original_moving_1.png')
showImg5_(MovingImage[z,:,:]).save('thesis/images/original_moving_histogram_matching_1.png')


In [ ]:
pathName = os.path.join(path, data[1])
'''
originalMovingImage = sitk.ReadImage(pathName)
MovingImage = sitk.HistogramMatching(sitk.CurvatureFlow(originalMovingImage,
                                                  timeStep = timeStep, 
                                                  numberOfIterations = numberOfIterations),
                               curvatureImage)
'''
originalMovingImage = sitk.CurvatureFlow(sitk.ReadImage(pathName),
                                                  timeStep = timeStep, 
                                                  numberOfIterations = numberOfIterations)
MovingImage = sitk.HistogramMatching(originalMovingImage, curvatureImage)

label = sitk.ReadImage(os.path.join(labelPath, data[1]))

t = registrationElastixMask(curvatureImage, MovingImage, label)
registeredImage = resampleImage(MovingImage,  curvatureImage, t)
registeredLabel = resampleLabels(label, curvatureImage, t)

In [ ]:
z = 60
image1, image2, image3 = createOverlays(originalLabel[z,:,:], registeredLabel[z,:,:])
showImg5_(image1).save('thesis/images/original_label_registration_1.png')
showImg5_(image2).save('thesis/images/moving_label_after_registration_1.png')
showImg5_(image3).save('thesis/images/combination_label_after_registration_1.png')
_, image2, image3 = createOverlays(originalLabel[z,:,:], label[z,:,:])
showImg5_(image2).save('thesis/images/moving_label_before_registration_1.png')
showImg5_(image3).save('thesis/images/combination_label_before_registration_1.png')

z = 250
image1, image2, image3 = createOverlays(originalLabel[:,z,:], registeredLabel[:,z,:])
showImg5_(image1[:,:,10:-120]).save('thesis/images/original_label_registration_2.png')
showImg5_(image2[:,:,10:-120]).save('thesis/images/moving_label_after_registration_2.png')
showImg5_(image3[:,:,10:-120]).save('thesis/images/combination_label_after_registration_2.png')
_, image2, image3 = createOverlays(originalLabel[:,z,:], label[:,z,:])
showImg5_(image2[:,:,10:-120]).save('thesis/images/moving_label_before_registration_2.png')
showImg5_(image3[:,:,10:-120]).save('thesis/images/combination_label_before_registration_2.png')

z = 250
image1, image2, image3 = createOverlays(originalLabel[:,:,z], registeredLabel[:,:,z])
showImg5_(image1[:,:,80:-80]).save('thesis/images/original_label_registration_3.png')
showImg5_(image2[:,:,80:-80]).save('thesis/images/moving_label_after_registration_3.png')
showImg5_(image3[:,:,80:-80]).save('thesis/images/combination_label_after_registration_3.png')
_, image2, image3 = createOverlays(originalLabel[:,:,z], label[:,:,z])
showImg5_(image2[:,:,80:-80]).save('thesis/images/moving_label_before_registration_3.png')
showImg5_(image3[:,:,80:-80]).save('thesis/images/combination_label_before_registration_3.png')

In [ ]:
showImg5_(image2[:,:,80:-80])

In [ ]:
image2.GetSize()

In [ ]:
z = 60
showImg5_(registeredImage[z,:,:])
showImg5_(curvatureImage[z,:,:])
showImg_(registeredLabel, z)
showImg_(originalLabel, z)

In [ ]:
# showImg5_(sitk.LabelOverlay(curvatureImage, originalLabel, opacity=0.1)[z,:,:])
# TODO: fix this, doesn't work
z = 60
# temp = sitk.LabelOverlay(curvatureImage, originalLabel, opacity=0.1)[z,:,:]
whiteImage = sitk.GetImageFromArray(255*np.ones(curvatureImage.GetSize()[::-1]))
whiteImage.CopyInformation(curvatureImage)
temp = sitk.LabelOverlay(whiteImage, originalLabel, opacity=0.1)[z,:,:]
whiteImage2 = sitk.GetImageFromArray(255*np.ones(registeredImage.GetSize()[::-1]))
whiteImage2.CopyInformation(registeredImage)
temp2 = sitk.LabelOverlay(whiteImage2, registeredLabel, opacity=0.1, colormap=(0,0,0,1,0,0,0,1))[z,:,:]

overlay1 = Image.fromarray(sitk.GetArrayFromImage(temp).astype('uint8'), 'RGB')
overlay2 = Image.fromarray(sitk.GetArrayFromImage(temp2).astype('uint8'), 'RGB')
overlay11 = overlay1.convert("RGBA")
overlay22 = overlay2.convert("RGBA")

new_img = Image.blend(overlay11, overlay22, 0.5)
# showImg5_(sitk.GetImageFromArray(new_img)[z,:,:])
showImg5_(sitk.GetImageFromArray(overlay11))
showImg5_(sitk.GetImageFromArray(overlay22))
showImg5_(sitk.GetImageFromArray(new_img))


In [ ]:
z = 60
image1, image2, image3 = createOverlays(originalLabel[z,:,:], registeredLabel[z,:,:])

showImg5_(image1)
showImg5_(image2)
showImg5_(image3)

In [ ]:
# image1, image2, image3 = createOverlays(originalLabel[z,:,:], registeredLabel[z,:,:])


In [ ]:
pathName = os.path.join(path, data[2])
originalMovingImage2 = sitk.CurvatureFlow(sitk.ReadImage(pathName),
                                                  timeStep = timeStep, 
                                                  numberOfIterations = numberOfIterations)
MovingImage2 = sitk.HistogramMatching(originalMovingImage2, curvatureImage)

label2 = sitk.ReadImage(os.path.join(labelPath, data[2]))

t = registrationElastixMask(curvatureImage, MovingImage2, label2)
registeredImage2 = resampleImage(MovingImage2,  curvatureImage, t)
registeredLabel2 = resampleLabels(label2, curvatureImage, t)

###############################################################################################

pathName = os.path.join(path, data[3])
originalMovingImage3 = sitk.CurvatureFlow(sitk.ReadImage(pathName),
                                                  timeStep = timeStep, 
                                                  numberOfIterations = numberOfIterations)
MovingImage3 = sitk.HistogramMatching(originalMovingImage3, curvatureImage)

label3 = sitk.ReadImage(os.path.join(labelPath, data[3]))

t = registrationElastixMask(curvatureImage, MovingImage3, label3)
registeredImage3 = resampleImage(MovingImage3,  curvatureImage, t)
registeredLabel3 = resampleLabels(label3, curvatureImage, t)

In [ ]:

z = 60
showImg5_(curvatureImage[z,130:270,80:220])
showImg5_(registeredImage[z,130:270,80:220])
showImg5_(registeredImage2[z,130:270,80:220])
showImg5_(registeredImage3[z,130:270,80:220])


In [ ]:
y_min, y_max = 130 + 60, 130 + 80
x_min, x_max = 80 + 50, 80 + 70
showImg5_(curvatureImage[z,y_min:y_max,x_min:x_max]).save('thesis/images/image_for_segmentation_patch.png')
showImg5_(registeredImage[z,y_min:y_max,x_min:x_max]).save('thesis/images/atlas_image_1_patch.png')
showImg5_(registeredImage2[z,y_min:y_max,x_min:x_max]).save('thesis/images/atlas_image_2_patch.png')
showImg5_(registeredImage3[z,y_min:y_max,x_min:x_max]).save('thesis/images/atlas_image_3_patch.png')

In [ ]:
from PIL import Image, ImageDraw

def to_image(img):
    a = sitk.GetArrayFromImage(img)
    return Image.fromarray(np.interp(a, (a.min(), a.max()), (0, 255)).astype('uint8')).convert('RGB')

def draw_rectangle(draw, coordinates, color, width=1):
    for i in range(width):
        rect_start = (coordinates[0][0] - i, coordinates[0][1] - i)
        rect_end = (coordinates[1][0] + i, coordinates[1][1] + i)
        draw.rectangle((rect_start, rect_end), outline = color)

def apply_rectangle(sitk_image, shape, color, width=5, show=True, draw_center=False):
    img = to_image(sitk_image)
    img1 = ImageDraw.Draw(img)  
    if draw_center:
        xx = (shape[0][0] + shape[1][0]) // 2
        yy = (shape[0][1] + shape[1][1]) // 2
        img1.rectangle([(xx-1, yy-1), (xx+1, yy+1)], fill=color, outline=color)
    
    draw_rectangle(img1, shape, color, width=width)
    if show:
        img.show()
    return img

def apply_rectangle_list(sitk_image, shape, color, rectangles, rectangles_width, width=5, show=True, draw_center=False):
    img = apply_rectangle(sitk_image, shape, color, width=width, show=False, draw_center=draw_center)
    img1 = ImageDraw.Draw(img)  
    
    for x, y in rectangles:
        # img1.rectangle([(x-rectangles_width, x-rectangles_width), (x+rectangles_width, x+rectangles_width)], fill='cyan', outline='cyan')
        img1.rectangle([(x-rectangles_width, y-rectangles_width), (x+rectangles_width, y+rectangles_width)], outline='cyan')
        
    if show:
        img.show()
    return img
        
    

In [ ]:
apply_rectangle(curvatureImage[z,130:270,80:220], [(60, 50), (80, 70)], 
                "green", draw_center=True).save('thesis/images/image_for_segmentation.png')
'''
apply_rectangle(registeredImage[z,130:270,80:220], [(60, 50), (80, 70)], "red")
apply_rectangle(registeredImage2[z,130:270,80:220], [(60, 50), (80, 70)], "red")
apply_rectangle(registeredImage3[z,130:270,80:220], [(60, 50), (80, 70)], "red")
'''

apply_rectangle(registeredImage[z,130:270,80:220], [(55, 45), (85, 75)], 
                "red").save('thesis/images/atlas_image_1.png')
apply_rectangle(registeredImage2[z,130:270,80:220], [(55, 45), (85, 75)], 
                "red").save('thesis/images/atlas_image_2.png')
apply_rectangle(registeredImage3[z,130:270,80:220], [(55, 45), (85, 75)], 
                "red").save('thesis/images/atlas_image_3.png')

In [ ]:
weights = [0.0, 0.0, 0.03251506760716438, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014015127904713154, 0.0, 0.0, 0.05124140903353691, 0.0014912447659298778, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.023481927812099457, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.032707054167985916, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2813958525657654, 0.0, 0.06913720816373825, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1080920621752739, 0.0, 0.0, 0.0, 0.0, 0.05280572175979614, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.059597447514534, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003153751837089658, 0.0]

import matplotlib.pyplot as plt

weightsFig, ax1 = plt.subplots(dpi=300)
ax1.set_xlim([0,len(weights)-1])
plt.plot(weights)
plt.show()


weightsFig.savefig('thesis/images/weights_plot.png')


In [ ]:
z = 60
apply_rectangle(curvatureImage[z,130:270,80:220], [(69, 59), (71, 61)], 
                "red", draw_center=True).save('thesis/images/segmentation_voxel.png')

In [ ]:
from random import randrange

xmin, xmax = 55, 85
ymin, ymax = 45, 75

rec_list = []
for _ in range(7):
    rec_list.append((randrange(xmin, xmax), randrange(ymin, ymax)))
apply_rectangle_list(registeredImage[z,130:270,80:220], [(55, 45), (85, 75)], 
                     "red", rec_list, 7, width=3).save('thesis/images/atlas_image_voxel_selection_1.png')

weights1 = np.zeros((30, 30), dtype='uint8')
for x, y in rec_list:
    weights1[y-ymin, x-xmin] = randrange(50, 255)
    for i in range(3):
        for j in range(3):
            xx = i - (x - xmin)
            yy = j - (y - ymin)
            if xx > 0 and yy > 0 and xx < xmax - xmin and yy < ymax - ymin:
                weights1[yy, xx] = weights1[y-ymin, x-xmin]
                
    for i in range(3):
        for j in range(3):
            xx = i + (x - xmin)
            yy = j + (y - ymin)
            if xx > 0 and yy > 0 and xx < xmax - xmin and yy < ymax - ymin:
                weights1[yy, xx] = weights1[y-ymin, x-xmin]
            
showImg5_(sitk.GetImageFromArray(weights1)).save('thesis/images/atlas_image_weights_1.png')

#########################################################################################################
rec_list = []
for _ in range(3):
    rec_list.append((randrange(xmin, xmax), randrange(ymin, ymax)))
apply_rectangle_list(registeredImage2[z,130:270,80:220], [(55, 45), (85, 75)], 
                     "red", rec_list, 7, width=3).save('thesis/images/atlas_image_voxel_selection_2.png')

weights1 = np.zeros((30, 30), dtype='uint8')
for x, y in rec_list:
    weights1[y-ymin, x-xmin] = randrange(50, 255)
    for i in range(3):
        for j in range(3):
            xx = i - (x - xmin)
            yy = j - (y - ymin)
            if xx > 0 and yy > 0 and xx < xmax - xmin and yy < ymax - ymin:
                weights1[yy, xx] = weights1[y-ymin, x-xmin]
                
    for i in range(3):
        for j in range(3):
            xx = i + (x - xmin)
            yy = j + (y - ymin)
            if xx > 0 and yy > 0 and xx < xmax - xmin and yy < ymax - ymin:
                weights1[yy, xx] = weights1[y-ymin, x-xmin]
            
showImg5_(sitk.GetImageFromArray(weights1)).save('thesis/images/atlas_image_weights_2.png')

#########################################################################################################
rec_list = []
for _ in range(1):
    rec_list.append((randrange(xmin, xmax), randrange(ymin, ymax)))
apply_rectangle_list(registeredImage3[z,130:270,80:220], [(55, 45), (85, 75)], 
                     "red", rec_list, 7, width=3).save('thesis/images/atlas_image_voxel_selection_3.png')

weights1 = np.zeros((30, 30), dtype='uint8')
for x, y in rec_list:
    weights1[y-ymin, x-xmin] = randrange(50, 255)
    for i in range(3):
        for j in range(3):
            xx = i - (x - xmin)
            yy = j - (y - ymin)
            if xx > 0 and yy > 0 and xx < xmax - xmin and yy < ymax - ymin:
                weights1[yy, xx] = weights1[y-ymin, x-xmin]
                
    for i in range(3):
        for j in range(3):
            xx = i + (x - xmin)
            yy = j + (y - ymin)
            if xx > 0 and yy > 0 and xx < xmax - xmin and yy < ymax - ymin:
                weights1[yy, xx] = weights1[y-ymin, x-xmin]
            
showImg5_(sitk.GetImageFromArray(weights1)).save('thesis/images/atlas_image_weights_3.png')

In [ ]:
test_img = to_image(curvatureImage[z,130:270,80:220])
img1 = ImageDraw.Draw(test_img)  
draw_rectangle(img1, [(60, 50), (80, 70)], "green", width=5)
test_img.show()

In [ ]:
test_img = to_image(curvatureImage[z,130:270,80:220])
img1 = ImageDraw.Draw(test_img)  
draw_rectangle(img1, [(60, 50), (80, 70)], "green", width=5)
test_img.show()

In [ ]:
apply_rectangle(registeredImage[z,130:270,80:220], [(55, 45), (85, 75)], "red", draw_center=True)